In [1]:
import pickle
#import gradio as gr
from combined_gui import predict

# Load binary_model to use
model_pkl = './models/second_binary_model.pkl'

# Load model from pickle file
with open(model_pkl, 'rb') as file:  
    model = pickle.load(file)  

# call the predict function
predict(model, (64, 60))

In [ ]:
# call the predict function
import pickle
from combined_gui import vid_predict

# Load binary_model to use
model_pkl = './models/second_binary_model.pkl'
  
# Load model from pickle file
with open(model_pkl, 'rb') as file:  
    model = pickle.load(file)  

vid_predict(model, (64, 60))